In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import norm

# --- 1. Setup ---
DATA_DIR = "data"
# Load individual asset returns
asset_returns = pd.read_csv(
    os.path.join(DATA_DIR, "asset_log_returns.csv"),
    index_col='Date',
    parse_dates=True
)
# Define current portfolio weights (equal weight) and confidence level
weights = np.array([1/asset_returns.shape[1]] * asset_returns.shape[1])
alpha = 0.01 # For 99% VaR

print("✅ Setup complete. Asset returns loaded.")

# --- 2. Calculate Portfolio Statistics ---
# Use the full historical covariance matrix and mean returns
cov_matrix = asset_returns.cov()
mean_returns = asset_returns.mean()
portfolio_mean_return = mean_returns.dot(weights)
portfolio_volatility = np.sqrt(weights.T @ cov_matrix @ weights)
z_score = norm.ppf(alpha)

# Calculate total Parametric VaR (expressed as a negative return)
total_var = portfolio_mean_return + portfolio_volatility * z_score

print(f"\nPortfolio Mean Return (Daily): {portfolio_mean_return*100:.4f}%")
print(f"Portfolio Volatility (Daily): {portfolio_volatility*100:.4f}%")
print(f"Z-score for {1-alpha:.0%} VaR: {z_score:.4f}")
print("-" * 40)
print(f"Total Portfolio 99% Parametric VaR: {total_var*100:.4f}%")
print("-" * 40)

✅ Setup complete. Asset returns loaded.

Portfolio Mean Return (Daily): 0.0446%
Portfolio Volatility (Daily): 1.0446%
Z-score for 99% VaR: -2.3263
----------------------------------------
Total Portfolio 99% Parametric VaR: -2.3856%
----------------------------------------


In [2]:
# Beta_i = Cov(Asset_i, Portfolio) / Var(Portfolio)
# Cov(Asset_i, Portfolio) is the i-th element of (cov_matrix @ weights)
beta_to_portfolio = (cov_matrix @ weights) / (portfolio_volatility**2)

print("--- Asset Betas to Portfolio ---")
print(beta_to_portfolio.round(4))

# --- 2. Calculate Marginal VaR (MVaR) ---
# We assume 'z_score' is in memory from sub-step 1
# MVaR_i = z_score * Cov(Asset_i, Portfolio) / Portfolio_Volatility
marginal_var = z_score * (cov_matrix @ weights) / portfolio_volatility

# Convert MVaR to percentage terms for easier interpretation
mvar_percent = marginal_var * 100

print("\n--- Marginal VaR (MVaR) ---")
print(mvar_percent.round(4).to_frame(name='MVaR (%)'))

--- Asset Betas to Portfolio ---
AAPL    1.3912
GE      1.4668
GLD     0.1215
HYG     0.4462
JPM     1.6646
MSFT    1.2925
NVDA    2.2099
QQQ     1.2354
SPY     1.1298
TLT    -0.2430
UUP    -0.0922
XLE     1.3772
dtype: float64

--- Marginal VaR (MVaR) ---
      MVaR (%)
AAPL   -3.3809
GE     -3.5647
GLD    -0.2952
HYG    -1.0844
JPM    -4.0452
MSFT   -3.1411
NVDA   -5.3706
QQQ    -3.0023
SPY    -2.7456
TLT     0.5906
UUP     0.2241
XLE    -3.3469
